In [1]:
import numpy as np
import math as math
import matplotlib.pyplot as plt

In [2]:
def create_data(n):
    '''
    returns data of n random points around centers (1,1) and (-1,-1) each
    points aroung (1,1): label 1
    points aroung (-1,-1): label 0
    '''
    
    c1 = np.random.normal(1, 0.5, (50, 2))
    c1 = np.hstack((c1, np.ones((50,1))))

    c2 = np.random.normal(-1, 0.5, (50, 2))
    c2 = np.hstack((c2, np.zeros((50,1))))

    data = np.vstack((c1, c2))
    return data

def sig_act(s):
    return 1.0/(1.0+math.exp(-s))

def sig_act_prime(s):
    return math.exp(-s)/(1.0+math.exp(-s))**2

def p_classifier(x1,x2,theta):
# compute 2-layer planar perceptron classifier for given inputs x1, x2
# parameter theta: a nine component vector

# change to weight, bias notation
    w11 = theta[0]; w12 = theta[1]; w21 = theta[2]; w22 =theta[3]
    w31 = theta[4]; w32 = theta[5]; b1 = theta[6]; b2=theta[7]; b3=theta[8]

    u1 = w11*x1 + w12*x2 + b1
    u2 = w21*x1 + w22*x2 + b2
      
    z1 = sig_act(u1)            # hidden node outputs
    z2 = sig_act(u2)
    u3 = w31*z1 + w32*z2 +b3    # 2nd layer
        
    
    out = sig_act(u3)         # output

    return(out)

def Loss(data, theta):
    '''returns Loss for provided separation line'''
    
    #calculate class prediction with specified line c_hat = sig(y - kx -d)
    loss = 0
    for i in range(len(data)):
        c_hat = p_classifier(data[i,0], data[i,1] ,theta)
        loss += (data[i,2] - c_hat)**2

    return loss

In [3]:
tdata = create_data(50)

In [4]:
def random_walk_metropolis_hastings(data, ntot, proposal_sd, theta_init):
    theta_save = np.zeros((ntot, 9))
    Loss_save = np.zeros(ntot)
    accept = np.zeros(9)

    theta_current = theta_init
    theta_prop = theta_current.copy()

    Loss_current = Loss(data, theta_current.tolist())
    
    for i in range(ntot):
        #store current state of MCMC chain
        theta_save[i,:] = theta_current
        Loss_save[i] = Loss_current
        
        for j in range(9):
            #draw a proposal for j-th component
            theta_prop[j] = np.random.normal(theta_current[j], proposal_sd, size = 1)

            #Loss_current = Loss(tdata, theta_current.tolist())
            #evaluate Loss of theta proposal state
            Loss_prop = Loss(data, theta_prop.tolist())
            
            #evaluate acceptance probability of proposal state
            #NOTICE: posterior = exp(-Loss(theta))
            #        for numerical stability in logarithm 
            alpha = np.log(np.exp(-Loss_prop)) - np.log(np.exp(-Loss_current))
            
            #update state with probability alpha
            u = np.random.uniform(size = 1)
            if alpha > np.log(u):
                theta_current = theta_prop.copy()
                Loss_current = Loss_prop
                accept[j] += 1
            else:
                theta_prop = theta_current.copy() #proposal was declined -> reset to current state
    
    return [theta_save, accept, Loss_save]

In [7]:
init = np.random.normal(size = (9))
theta_save, accept, Loss_save = random_walk_metropolis_hastings(data=tdata, ntot=20000, proposal_sd=1, theta_init=init)

OverflowError: math range error

In [ ]:
#Analysis of MCMC chain
print(f'acceptance rates: {accept/ntot}')
print(f'min Loss: {Loss_save.min()}')
print(f'mean Loss: {Loss_save.mean()}')
print(f'max Loss: {Loss_save.max()}')
print(f'std Loss: {Loss_save.std()}')

In [ ]:
#Convergence diagnostics - Trace plots
import pandas as pd
pd.DataFrame(theta_save).plot(subplots = True)

In [ ]:
nburn = 5000
n = ntot - nburn

fig, axs = plt.subplots(3,3, sharey='row')

for i, ax in enumerate(axs.flatten()):
    aa1,bb1 = np.histogram(theta_save[-n:,i], bins=50,density=True)
    
    ax.plot(bb1[:-1],aa1)
    ax.set_xlabel(r'$\theta$'+f'$_{i}$')
    #ax.set_ylabel('density')

#plt.title(f"Marginal posterior of theta[{pos}]")
plt.tight_layout()
plt.show()